In [1]:
import os
from typing import Callable, List, Optional, Tuple

import attr
import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as scipy_img
import scipy.interpolate as sci
from cv2 import VideoWriter, VideoWriter_fourcc

In [109]:
width = 640
height = 480
FPS = 15
seconds = 5 # 30
frames = int(FPS * seconds)

# Could use "test.mp4" and the "H264" fourcc codec below.
FILENAME = "test.webm"
if os.path.exists(FILENAME):
    os.remove(FILENAME)

fourcc = VideoWriter_fourcc(*"VP80")
video = VideoWriter(FILENAME, fourcc, float(FPS), (width, height))

mod0 = 0.0
#theta = 0.0
max_radius = 0.5 * (width / 2)
circle_radius = 1
offset_x = 320
offset_y = 240

# Modulate the persistance decay between these two values:
decay_max = 0.08
decay_min = 0.03

# If True, "zoom" the image so that the persistence is scaled up each frame.
zoom = True

ncircles = 250
radii = np.random.random((ncircles,)) * max_radius
thetas = np.random.random((ncircles,)) * (np.pi * 2.0)

radii_mod = np.random.random((ncircles,)) * (np.pi * 2.0)

# Factor in the radius in the theta delta, so that outer circles don't move too quickly.
thetas_delta = np.pi / (radii + np.random.random((ncircles,)) * 0.5)

persistent = np.zeros((height, width, 3))
persistent2 = np.zeros((height, width, 3))

tone = (190, 255, 255)

for frame_num in range(frames):
    frame = np.zeros((height, width, 3), dtype=np.uint8)
    
    eff_radii = radii + (2.5 * radii * np.sin(radii_mod))
    xs = np.round((np.sin(thetas) * eff_radii) + offset_x).astype(int)
    ys = np.round((np.cos(thetas) * eff_radii) + offset_y).astype(int)
    
    for i in range(ncircles):
        x = xs[i]
        y = ys[i]
        cv2.circle(img=frame, center=(x, y), radius=circle_radius, color=tone, thickness=-1, lineType=cv2.LINE_8)
    
    mod_slow = np.sin((FPS / 100.0) * 2.0 * np.pi * mod0)
    # Sin yields +/- 0.5 -- get it into 0-1 range.
    mod_slow_norm = mod_slow + 0.5
    # Now scale to (decay_min, decay_max)
    decay = (mod_slow_norm * (decay_max - decay_min)) + decay_min
    
    # Apply a heavy gaussian to one of the persistent layers.
    kernel = 51
    persistent = cv2.GaussianBlur(persistent, (kernel, kernel), 0.0)
    persistent = np.clip((persistent + frame) * (1.0 - decay), 0, 255)
    
    # And a lighter one to the second persistent layer.
    kernel = 11
    persistent2 = cv2.GaussianBlur(persistent2, (kernel, kernel), 0.0)
    persistent2 = np.clip((persistent2 + frame) * (1.0 - decay), 0, 255)
    
    if zoom:
        # slightly scale up persistency layers each tick
        zoom_by = 40
        zb_half = zoom_by // 2
        persistent = cv2.resize(persistent, (width + zoom_by, height + zoom_by))[zb_half:-zb_half, zb_half:-zb_half]
        persistent2 = cv2.resize(persistent2, (width + zoom_by, height + zoom_by))[zb_half:-zb_half, zb_half:-zb_half]
    
    # TODO: It's inefficient to create 3 channels of data; can make do with one.
    # TODO: Could map to HSV and use V as the color map input.
    persistent_channel = np.clip((persistent + persistent2)[:, :, 0], 0, 255).astype(np.uint8)
    
    # Map one of the channels to a colormap; can also use cv2.COLORMAP_OCEAN for a blue, cool aesthetic.
    persistent_combined = cv2.applyColorMap(persistent_channel, cv2.COLORMAP_HOT)
    combined = np.clip(persistent_combined, 0, 255).astype(np.uint8)
    
    # Finally, write out the frame.
    video.write(combined)
    
    # Increment all modulated parameters
    thetas += thetas_delta / FPS
    
    mod0 += np.pi / (12 * FPS)
    radii_mod += np.pi / (12 * FPS)

video.release()

In [ ]:
from IPython.display import Video

Video(FILENAME, embed=True)